In [15]:
import pandas as pd
import os
import numpy as np

In [16]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\SQL---Coderhouse---Proyecto-Final\\csvs\\originales')


In [17]:
books = pd.read_csv('books_data.csv')
ratings = pd.read_csv('Books_rating.csv')

In [ ]:
'''Datos originilales
Books (212404,10)
ratings(30000000,10)'''

In [18]:
#Aca va el proceso de normalizacion de las tablas
#Eliminar columnas
books = books.drop(columns=['image','previewLink','infoLink','ratingsCount', 'description'])
ratings = ratings.drop(columns=['review/helpfulness','review/time', 'review/summary', 'review/text'])
#merge de ambas columnas, a fin de eliminar valores y no dejar datos huerfanos
merged_df = pd.merge(books,ratings, on='Title')
merged_df = merged_df.dropna(subset=['Title']) #elimino filas donde 'Title' es NaN value
merged_df = merged_df.dropna(subset=['User_id', 'profileName'], how='all') #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['profileName']) #Elimino Nan Values relacionado con usuarios
merged_df = merged_df.dropna(subset=['publisher', 'publishedDate', 'categories']) #elimino mas Nan values
merged_df = merged_df.dropna(subset=['authors'])#elimino mas Nan values

In [19]:
merged_df.columns

Index(['Title', 'authors', 'publisher', 'publishedDate', 'categories', 'Id',
       'Price', 'User_id', 'profileName', 'review/score'],
      dtype='object')

In [20]:
'''CASO COLUMNA PRICE
Existe una cantidad enorme de Nan Values, pero algunos son recuperables, ya que segun el nro de ID, algunos books tiene precio,
entonces,procedo a completar esos valores que se pueden rellenar'''

# Calcular el valor promedio de Prices para cada book_id
avg_prices = merged_df.groupby('Id')['Price'].mean()

# Rellenar los valores NaN en Prices con el valor promedio para el book_id correspondiente
merged_df['Price'] = merged_df.apply(lambda row: avg_prices[row['Id']] if pd.isna(row['Price']) else row['Price'], axis=1)

#los prices que siguen quedando como Nan, se descartan
merged_df = merged_df.dropna(subset=['Price'])#elimino mas Nan values

'''De esta manera, queda un dataset con 306450 filas por 11 columnas
al cual hay que dividir en 5 tablas'''

'De esta manera, queda un dataset con 306450 filas por 11 columnas\nal cual hay que dividir en 5 tablas'

In [21]:
# genero nros de id desde 1
def add_factorize_id_column(df, column_name, new_column_name):
    df[new_column_name] = pd.factorize(df[column_name])[0] + 1
    return df

merged_df = add_factorize_id_column(merged_df, 'Title', 'id_book')
merged_df = add_factorize_id_column(merged_df, 'User_id', 'id_user')
merged_df = add_factorize_id_column(merged_df, 'categories', 'id_categories')

#Elimino mas columnas que no sirven
merged_df = merged_df.drop(columns=['Id', 'User_id'])

In [22]:
#Limpio las columnas de caracteres no deseados
def clean_column(col):
    col = col.str.replace('"', '').str.replace("'", '')
    col = col.str.strip('[]')
    col = col.str.replace('"', '').str.replace("'", '')
    return col

merged_df[['profileName', 'authors', 'categories']] = merged_df[['profileName', 'authors', 'categories']].apply(clean_column)

# Extrae solo el año de la columna 'publishedDate'
merged_df['publishedDate'] = merged_df['publishedDate'].str.extract(r'\b(\d{4})\b')

#cambio el nombre de las columnas
merged_df = merged_df.rename(columns={'Title': 'title', 'publishedDate': 'published_year', 'Price': 'price',
                                      'profileName': 'profile_name', 'review/score': 'score'})

#reinicio el indice
merged_df = merged_df.reset_index(drop=True)

In [23]:
#Separar merged_df en 5 tablas diferentes
def create_table(df, cols, drop_duplicates=None):
    table = df[cols]
    if drop_duplicates:
        table = table.drop_duplicates(subset=drop_duplicates)
    table = table.reset_index(drop=True)
    return table

books = create_table(merged_df, ['id_book', 'title', 'authors', 'publisher', 'published_year', 'id_categories'], 'id_book')
ratings = create_table(merged_df, ['id_user', 'id_book', 'score'])
categories = create_table(merged_df, ['id_categories', 'categories'], 'id_categories')
prices = create_table(merged_df, ['id_book', 'price'], 'id_book')
users = create_table(merged_df, ['id_user', 'profile_name'], 'id_user')


In [24]:
#ultimos arreglos en tabla books:
#quedaron 2 filas con valores null
books = books.dropna()
books['published_year'] = books['published_year'].astype(int) #paso published_year de object a int

In [10]:
books

,id_book,title,authors,publisher,published_year,id_categories
0,1,Whispers of the Wicked Saints,Veronica Haddon,iUniverse,2005,1
1,2,The Church of Christ: A Biblical Ecclesiology ...,Everett Ferguson,Wm. B. Eerdmans Publishing,1996,2
2,3,Saint Hyacinth of Poland,Mary Fabyan Windeatt,Tan Books & Pub,2009,3
3,4,Vector Quantization and Signal Compression (Th...,"Allen Gersho, Robert M. Gray",Springer Science & Business Media,2012,4
4,5,The Ultimate Guide to Law School Admission: In...,Fiona Cownie,Bloomsbury Publishing,2010,5
...,...,...,...,...,...,...
33908,33909,Governance Of Science (Issues in Society),"Fuller, Steve",McGraw-Hill Education (UK),1999,22
33909,33910,Is Your Retirement Heading in the Right Direct...,Steven Casto,"Strategic Wealth Solutions, LLC",2005,16
33910,33911,The Awakening and Selected Stories (Modern Lib...,Kate Chopin,Library of Alexandria,2003,517
33911,33912,The Orphan Of Ellis Island (Time Travel Advent...,Elvira Woodruff,Scholastic Paperbacks,2000,26


In [13]:
#Dimensiones de las tablas una vez finalizado el proceso
print('books: ', books.shape)
print('reviews: ', reviews.shape)
print('categories: ', categories.shape)
print('prices: ', prices.shape)
print('users: ', users.shape)

books:  (33911, 6)
reviews:  (306450, 3)
categories:  (942, 2)
prices:  (33913, 2)
users:  (236353, 2)


In [14]:
#herramientas de consulta:
null_counts = merged_df.isnull().sum()
print(null_counts) #ver cuantos Nan values hay en cada columna
print('----------------')
print(merged_df.nunique())#Ver cuantos valores unicos hay por columna
print('----------------')
print(merged_df['Price'].describe())
print('----------------')
print(merged_df.shape)
print('----------------')

'''Use este codigo para ver cuantos prices pueden completarse con valores ya existentes
def has_mixed_prices(group):
    has_nan = np.isnan(group['Price']).any()
    has_number = np.isfinite(group['Price']).any()
    return has_nan and has_number

has_mixed = merged_df.groupby('id_book').apply(has_mixed_prices)
book_ids_with_mixed_prices = has_mixed[has_mixed].index.to_list()
print("Book IDs con precios mixtos:", book_ids_with_mixed_prices)'''

title             0
authors           0
publisher         0
published_year    8
categories        0
price             0
profile_name      0
score             0
id_book           0
id_user           0
id_categories     0
dtype: int64
----------------
title              33913
authors            29984
publisher           5644
published_year       124
categories           942
price               5224
profile_name      213344
score                  5
id_book            33913
id_user           236353
id_categories        942
dtype: int64
----------------


KeyError: 'Price'

In [25]:
os.chdir('C:\\Users\\juanm\\OneDrive\\Escritorio\\pythonProject\\SQL---Coderhouse---Proyecto-Final\\csvs\\normalizados')

In [26]:
books.to_csv('books.csv', index=False)
ratings.to_csv('ratings.csv', index=False)
categories.to_csv('categories.csv', index=False)
prices.to_csv('prices.csv', index=False)
users.to_csv('users.csv', index=False)
